# Machine Learning Volatility Modeling
## Master's Thesis - Empirical Study 
### Università della Svizzera italiana

Pietro Bonazzi - pietro.bonazzi@usi.ch

Data Processing - v2.2

In [98]:
import pandas as pd 
import numpy as np
import sqlite3

In [99]:
# Read data from dataframe SQL
conn = sqlite3.connect("data.db")
df = pd.read_sql_query("SELECT * FROM price", conn)

In [102]:
# Check tables in SQL database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = pd.DataFrame(cursor.fetchall()).transpose()
tables

,0,1,2,3
0,price,returns,rv,rq


In [103]:
# Cleaner tool (temp)

tables_to_keep = ['price', 'returns', 'rv', 'rq']

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    if table_name not in tables_to_keep:
        cursor.execute(f"DROP TABLE `{table_name}`")
        
conn.commit()


### Create an SQL table for each stock

In [104]:
# Read data from dataframe SQL

# Returns resampled to daily frequency
df_prices = pd.read_sql_query("SELECT * FROM price", conn)
df_prices.index = pd.to_datetime(df_prices['event_timestamp'])
df_prices = df_prices.drop(columns=['event_timestamp','index'])
df_prices_resampled = df_prices.groupby(df_prices.index.date).last()
df_returns_resampled = np.log(df_prices_resampled) - np.log(df_prices_resampled.shift(1))
df_returns_resampled.dropna(inplace=True)
df_returns_resampled.index.name = 'date'

# Realized volatility
df_rv = pd.read_sql_query("SELECT * FROM rv", conn)
df_rv.index = pd.to_datetime(df_rv['index'])
df_rv.drop(columns=['index'], inplace=True)
df_rv.index.name = 'date'

# Realized quarticity
df_rq = pd.read_sql_query("SELECT * FROM rq", conn)
df_rq.index = pd.to_datetime(df_rq['index'])
df_rq.drop(columns=['index'], inplace=True)
df_rq.index.name = 'date'

# Additional Covariates
stock_covariates = pd.read_csv('raw_data/additional_stock_covariates.csv', index_col=0, sep=",")
stock_covariates.index.name = 'date'
stock_covariates.index = pd.to_datetime(stock_covariates.index, format='%d/%m/%y')

In [108]:
# Define additional covariates
covariates = ['PX_VOLUME',                      # Number of shares traded
              'AVERAGE_BID_ASK_SPREAD_%',       # Average bid-ask spread
              'RSK_BB_IMPLIED_CDS_SPREAD',      # Implieed CDS spread
              'HIST_PUT_IMP_VOL',               # Historical put implied volatility
              'NEWS_SENTIMENT_DAILY_AVG']       # News sentiment (BBG machine learning model, value range [-1,1])

# Combine dataframes and save to SQL database
for stock in df_rv.columns:
    df_combined = pd.concat([df_returns_resampled[stock],
                                df_rv[stock], 
                                df_rq[stock], 
                                stock_covariates[stock + '_' + covariates[0]],
                                stock_covariates[stock + '_' + covariates[1]],
                                stock_covariates[stock + '_' + covariates[2]],
                                stock_covariates[stock + '_' + covariates[3]],
                                stock_covariates[stock + '_' + covariates[4]]
                                ], axis=1)
    df_combined.columns = ['Returns','RV', 'RQ', covariates[0], covariates[1], covariates[2], covariates[3], covariates[4]]
    df_combined.dropna(inplace=True)
    df_combined.to_sql(stock, conn, if_exists='replace')

In [120]:
mkt_covariates = pd.read_csv('raw_data/additional_mkt_covariates.csv', index_col=0, sep=",")
mkt_covariates.index.name = 'date'
mkt_covariates.index = pd.to_datetime(mkt_covariates.index, format='%d/%m/%y')
mkt_covariates.dropna(inplace=True)
mkt_covariates.to_sql('mkt_covariates', conn, if_exists='replace')

vsmi = pd.read_csv('raw_data/vsmi.csv', index_col=0, sep=";")
vsmi.index.name = 'date'
vsmi.columns = ['ISIN', 'VSMI']
vsmi.index = pd.to_datetime(vsmi.index, format='%d.%m.%Y')
vsmi = vsmi.loc['2014-12-15':'2023-06-30']
vsmi.drop(columns=['ISIN'], inplace=True)

pd.concat([vsmi, mkt_covariates], axis=1).to_sql('mkt_covariates', conn, if_exists='replace')

2230

### Test retrieving data from SQL database

In [122]:
# Check tables in SQL database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = pd.DataFrame(cursor.fetchall()).transpose()
tables

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,price,returns,rv,rq,ABBN,CFR,GEBN,GIVN,HOLN,KNIN,...,NOVN,PGHN,ROG,SCMN,SIKA,SOON,SREN,UBSG,ZURN,mkt_covariates


In [125]:
# Testing reading data from dataframe SQL
df = pd.read_sql_query("SELECT * FROM UBSG", conn)
df.index = pd.to_datetime(df['date'])
df.drop(columns=['date'], inplace=True)
df

,Returns,RV,RQ,PX_VOLUME,AVERAGE_BID_ASK_SPREAD_%,RSK_BB_IMPLIED_CDS_SPREAD,HIST_PUT_IMP_VOL,NEWS_SENTIMENT_DAILY_AVG
date,,,,,,,,
2014-12-16,0.001197,0.016156,0.000057,14606852,0.0759,75.9,24.110,0.0114
2014-12-17,0.002987,0.010568,0.000037,8231113,0.0738,76.5,23.620,0.0735
2014-12-18,0.019493,0.014132,0.000135,10845548,0.0669,76.2,20.148,-0.1462
2014-12-19,0.001753,0.023170,0.000401,29634438,0.0689,76.4,20.863,0.4073
2014-12-22,0.002915,0.009102,0.000023,6980144,0.0696,75.9,20.033,0.4073
...,...,...,...,...,...,...,...,...
2023-06-26,-0.010353,0.020653,0.000201,5631210,0.0285,81.4,23.565,-0.0026
2023-06-27,-0.001594,0.015471,0.000070,4271836,0.0278,81.3,23.774,0.0000
2023-06-28,0.012734,0.017731,0.000236,5159563,0.0271,81.2,22.274,0.0054


In [96]:
conn.close()